<a href="https://colab.research.google.com/github/SHEHAN-120/collaborative-book-recommendation/blob/main/book_recommended_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Book Recommended System

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data={
    'user_id':[1,1,1,2,2,3,3,3,4,4],
    'book_title': [
        'Book A','Book B','Book C','Book A','Book D',
        'Book B','Book C','Book E','Book A','Book C'
    ],
    'rating': [5,3,4,4,5,5,3,4,3,2]
}

In [3]:
df=pd.DataFrame(data)
print("Dataset:\n",df)

Dataset:
    user_id book_title  rating
0        1     Book A       5
1        1     Book B       3
2        1     Book C       4
3        2     Book A       4
4        2     Book D       5
5        3     Book B       5
6        3     Book C       3
7        3     Book E       4
8        4     Book A       3
9        4     Book C       2


In [4]:
user_book_matrix=df.pivot_table(index='user_id',columns='book_title',values='rating').fillna(0)
print("\nUser-Book Matrix:\n",user_book_matrix)


User-Book Matrix:
 book_title  Book A  Book B  Book C  Book D  Book E
user_id                                           
1              5.0     3.0     4.0     0.0     0.0
2              4.0     0.0     0.0     5.0     0.0
3              0.0     5.0     3.0     0.0     4.0
4              3.0     0.0     2.0     0.0     0.0


### Calculate Cosine Similarity

In [5]:
user_similarity=cosine_similarity(user_book_matrix)
user_similarity_df=pd.DataFrame(user_similarity,index=user_book_matrix.index,columns=user_book_matrix.index)
print("\nUser Similarity Matrix:\n",user_similarity_df)


User Similarity Matrix:
 user_id         1         2         3         4
user_id                                        
1        1.000000  0.441726  0.540000  0.902134
2        0.441726  1.000000  0.000000  0.519778
3        0.540000  0.000000  1.000000  0.235339
4        0.902134  0.519778  0.235339  1.000000


In [6]:
def recommend_books(user_id,similarity_matrix,user_book_matrix,top_n=3):
  if user_id not in similarity_matrix.index:
    print("User not found in the dataset")
    return[]

  similar_users=similarity_matrix[user_id].sort_values(ascending=False).drop(user_id)

  recommended_books={}
  for sim_user,similarity in similar_users.items():
    rated_books=user_book_matrix.loc[sim_user]
    for book,rating in rated_books[rated_books>0].items():
      if book not in user_book_matrix.loc[user_id] or user_book_matrix.loc[user_id,book]==0:
        recommended_books[book]=recommended_books.get(book,0) + rating * similarity

  recommended_books=sorted(recommended_books.items(),key=lambda x:x[1],reverse=True)
  return [book for book,score in recommended_books[:top_n]]




In [7]:
user_id=3
recommended_books=recommend_books(user_id,user_similarity_df,user_book_matrix,top_n=3)
print(f"\nRecommended Books for User {user_id}:", recommended_books)





Recommended Books for User 3: ['Book A', 'Book D']
